In [1]:
import pandas as pd
import numpy as np
import pyproj

In [3]:
# read 전체 데이터
df = pd.read_csv('./fulldata_07_24_04_P_일반음식점.csv',encoding='cp949')
len(df)

/var/folders/_2/tym68nn973j_g5h4jw45p8180000gn/T/ipykernel_60415/488824028.py:2: DtypeWarning: Columns (22,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./fulldata_07_24_04_P_일반음식점.csv',encoding='cp949')


2012833

In [4]:
# 폐업 필터링
mask = df.영업상태명 != "폐업"
df = df.loc[mask,:]

# 좌표 데이터 타입 numberic으로 변경
df['좌표정보(x)'] = pd.to_numeric(df['좌표정보(x)'], errors="coerce")
df['좌표정보(y)'] = pd.to_numeric(df['좌표정보(y)'], errors="coerce")

len(df)

689698

In [5]:

def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 26], coord[:, 27])
    return np.dstack([fx, fy])[0]


# DataFrame -> NumPy Array 변환
coord = np.array(df)

# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)


df['좌표정보(x)'] = result[:, 1]
df['좌표정보(y)'] = result[:, 0]

/Users/kukim/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/kukim/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/_2/tym68nn973j_g5h4jw45p8180000gn/T/ipykernel_60415/2306189217.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.g

In [6]:
data = df.iloc[:, [15, 19, 21, 26, 27]]
data.columns = ['contactNumber','address','storeName','latitude', 'longitude']
data = data[data.address.apply(lambda x: len(str(x).split(",")) == 2)]
data['baseAddress'] = data.address.apply(lambda x: str(x).split(",")[0])
data['detailAddress'] = data.address.apply(lambda x: str(x).split(",")[1])

len(data)

387351

In [7]:
filtered_df=data.loc[:,data.columns!="address"]

filtered_df.to_csv('./done.csv',index = False, header=True)

In [8]:

filtered_df.head()

,contactNumber,storeName,latitude,longitude,baseAddress,detailAddress
1323135,NaN,명품스테이크,37.442487,127.146131,경기도 성남시 수정구 산성대로295번길 9,1(일부)층 (신흥동)
1323136,NaN,79대포 안성공도점,37.000065,127.166656,경기도 안성시 공도읍 공도3로 9-9,공도 일번가타워 1층 107호
1323137,NaN,맛있어2(Masisso2),37.001180,127.220998,경기도 안성시 대덕면 중앙대학로 116-1,2층
1323138,NaN,푸드 딜리버리 영등포점,37.509676,126.904280,서울특별시 영등포구 도신로33길 11,1층 (도림동)
1323139,NaN,수벙샌드위치,35.306876,128.985118,경상남도 양산시 물금읍 버들2길 7-13,1층 일부
